In [2]:
import pandas as pd
from metaphone import doublemetaphone

In [22]:
def read_kaggle_names():
    dfa = pd.read_csv("resources/kaggle_female_names.csv")
    dfm = pd.read_csv("resources/kaggle_male_names.csv")    
    dfr = pd.concat([dfa, dfm])[["name"]]
    dfr["TYPE"] = "NAME"
    return dfr

def f_dmetaphone(x):
    try:
        return doublemetaphone(x)[0]
    except Exception as ex:
        print(f"Exception for {x}:", ex)
        return None

#print("doublemetaphone(x)[0]:", doublemetaphone("")[0])
dfk = read_kaggle_names()
dfk.dropna()
dfk["METAPHONE"] = dfk["name"].apply(lambda x : f_dmetaphone(x))
dfk.head()

Exception for nan: 'float' object has no attribute 'replace'


,name,TYPE,METAPHONE
0,MARIA CARMEN,NAME,MRKRMN
1,MARIA,NAME,MR
2,CARMEN,NAME,KRMN
3,JOSEFA,NAME,JSF
4,ISABEL,NAME,ASPL


In [27]:
def read_ine_names():
    dfmn = pd.read_csv("resources/ine_male_names.csv")
    dfmn = dfmn[["name"]]
    dffn = pd.read_csv("resources/ine_female_names.csv")
    dffn = dffn[["name"]]
    dfr = pd.concat([dfmn, dffn])
    dfr["TYPE"] = "NAME"
    
    dfms = pd.read_csv("resources/ine_surnames_freq_ge_100.csv")
    dfms = dfms[["surname"]]
    dfms["TYPE"] = "SURNAME"
    dfms.rename(columns = {'surname': 'name'}, inplace = True)
    
    dfr = pd.concat([dfms, dfr])
    return dfr

dfi = read_ine_names()
dfi["METAPHONE"] = dfi["name"].apply(lambda x : f_dmetaphone(x))
dfi[dfi["TYPE"] != "SURNAME"].head()

Exception for nan: 'float' object has no attribute 'replace'


,name,TYPE,METAPHONE
0,ANTONIO,NAME,ANTN
1,JOSE,NAME,JS
2,MANUEL,NAME,MNL
3,FRANCISCO,NAME,FRNSSK
4,JUAN,NAME,JN


In [28]:
dfTarget = pd.concat([dfi, dfk]).drop_duplicates()
dfTarget.head()

,name,TYPE,METAPHONE
0,GARCIA,SURNAME,KRS
1,GONZALEZ,SURNAME,KNSLS
2,RODRIGUEZ,SURNAME,RTRKS
3,FERNANDEZ,SURNAME,FRNNTS
4,LOPEZ,SURNAME,LPS


In [29]:
def read_db_dataset():
    dfdb = pd.read_csv("resources/export_db.csv")
    return dfdb

dfdbInput = read_db_dataset()
dfdbInput.head()

,NAME,SURNAME
0,AMPARO DEL ROSARIO,DE LA ESPRIELLA HOYOS
1,EDGARD,SALAS CORREA
2,LINO,BETANCOURT
3,L.,DE FONSECA
4,KATIA,GIRADO LLMAS


In [30]:
from fuzzywuzzy import fuzz, process

In [69]:
def get_relative_names(x):
    vDoubleMetaphone = process.extractOne(f_dmetaphone(x), dfTarget['METAPHONE'])[0]
    dfResult = dfTarget[dfTarget["METAPHONE"] == vDoubleMetaphone]
    return dfResult

def get_predicted_name(x):
    vNames = get_relative_names(x)
    vResult = process.extract(x, vNames["name"])
    print(f"Similar metaphones for {x} are: ", vResult)
    vResult = process.extractOne(x, vNames["name"])
    print(f"Predicted correct for {x} is: ", vResult)
    return vNames

In [72]:
vNames = get_predicted_name("Jjhohaan")
vNames.head(10)

Similar metaphones for Jjhohaan are:  [('JOHAN', 77, 2241), ('JOHANN', 71, 3460), ('JOHANA', 71, 1340), ('JOHANNY', 67, 22344), ('JOHANNA', 67, 1064)]
Predicted correct for Jjhohaan is:  ('JOHAN', 77, 2241)


,name,TYPE,METAPHONE
2241,JOHAN,NAME,JHN
3460,JOHANN,NAME,JHN
22344,JOHANNY,NAME,JHN
1064,JOHANNA,NAME,JHN
1340,JOHANA,NAME,JHN
2311,JIHANE,NAME,JHN
2844,JIHAN,NAME,JHN
20339,ZHIHAN,NAME,JHN
20625,GIHAN,NAME,JHN
20681,JEHAN,NAME,JHN


In [77]:
vNames = get_predicted_name("jojan")
vNames.head(10)

Similar metaphones for jojan are:  [('GIJON', 60, 1515), ('JIAJUN', 55, 4495), ('ZHIJIAN', 50, 12994), ('JIA JUN', 50, 13127), ('ZHIJUN', 36, 20483)]
Predicted correct for jojan is:  ('GIJON', 60, 1515)


,name,TYPE,METAPHONE
1515,GIJON,SURNAME,JJN
4495,JIAJUN,NAME,JJN
12994,ZHIJIAN,NAME,JJN
13127,JIA JUN,NAME,JJN
20483,ZHIJUN,NAME,JJN
